In [28]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [6]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_text
from ucimlrepo import fetch_ucirepo
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [7]:

breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets
print(breast_cancer_wisconsin_diagnostic.metadata)
print(breast_cancer_wisconsin_diagnostic.variables)



{'uci_id': 17, 'name': 'Breast Cancer Wisconsin (Diagnostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic', 'data_url': 'https://archive.ics.uci.edu/static/public/17/data.csv', 'abstract': 'Diagnostic Wisconsin Breast Cancer Database.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 569, 'num_features': 30, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Diagnosis'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1993, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5DW2B', 'creators': ['William Wolberg', 'Olvi Mangasarian', 'Nick Street', 'W. Street'], 'intro_paper': {'ID': 230, 'type': 'NATIVE', 'title': 'Nuclear feature extraction for breast tumor diagnosis', 'authors': 'W. Street, W. Wolberg, O. Mangasarian', 'venue': 'Electronic imaging', 'year': 1993, 'journal': None, 'DOI': '1

In [8]:
# 打印每一列的索引和列名
print("Column Index and Name:")
for idx, column_name in enumerate(X.columns):
    print(f"Index: {idx}, Column Name: {column_name}")


Column Index and Name:
Index: 0, Column Name: radius1
Index: 1, Column Name: texture1
Index: 2, Column Name: perimeter1
Index: 3, Column Name: area1
Index: 4, Column Name: smoothness1
Index: 5, Column Name: compactness1
Index: 6, Column Name: concavity1
Index: 7, Column Name: concave_points1
Index: 8, Column Name: symmetry1
Index: 9, Column Name: fractal_dimension1
Index: 10, Column Name: radius2
Index: 11, Column Name: texture2
Index: 12, Column Name: perimeter2
Index: 13, Column Name: area2
Index: 14, Column Name: smoothness2
Index: 15, Column Name: compactness2
Index: 16, Column Name: concavity2
Index: 17, Column Name: concave_points2
Index: 18, Column Name: symmetry2
Index: 19, Column Name: fractal_dimension2
Index: 20, Column Name: radius3
Index: 21, Column Name: texture3
Index: 22, Column Name: perimeter3
Index: 23, Column Name: area3
Index: 24, Column Name: smoothness3
Index: 25, Column Name: compactness3
Index: 26, Column Name: concavity3
Index: 27, Column Name: concave_points3

In [9]:
print(X.head())
print(y[:10])  

   radius1  texture1  perimeter1   area1  smoothness1  compactness1  \
0    17.99     10.38      122.80  1001.0      0.11840       0.27760   
1    20.57     17.77      132.90  1326.0      0.08474       0.07864   
2    19.69     21.25      130.00  1203.0      0.10960       0.15990   
3    11.42     20.38       77.58   386.1      0.14250       0.28390   
4    20.29     14.34      135.10  1297.0      0.10030       0.13280   

   concavity1  concave_points1  symmetry1  fractal_dimension1  ...  radius3  \
0      0.3001          0.14710     0.2419             0.07871  ...    25.38   
1      0.0869          0.07017     0.1812             0.05667  ...    24.99   
2      0.1974          0.12790     0.2069             0.05999  ...    23.57   
3      0.2414          0.10520     0.2597             0.09744  ...    14.91   
4      0.1980          0.10430     0.1809             0.05883  ...    22.54   

   texture3  perimeter3   area3  smoothness3  compactness3  concavity3  \
0     17.33      184.60 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")


Training data shape: (455, 30)
Test data shape: (114, 30)


In [11]:
clf = DecisionTreeClassifier(
    criterion='gini', 
    min_samples_leaf=2,  
    min_samples_split=5,  
    max_depth=2, 
    random_state=42
)

clf.fit(X_train, y_train)

accuracy = clf.score(X_train, y_train)
print(f"Training Accuracy: {accuracy:.10f}")



Training Accuracy: 0.9296703297


In [12]:
tree = export_text(clf, feature_names=list(X.columns))
print(tree)

|--- concave_points1 <= 0.05
|   |--- radius3 <= 16.83
|   |   |--- class: B
|   |--- radius3 >  16.83
|   |   |--- class: M
|--- concave_points1 >  0.05
|   |--- concave_points3 <= 0.15
|   |   |--- class: B
|   |--- concave_points3 >  0.15
|   |   |--- class: M



In [118]:
tree = clf.tree_
features = X.columns

first_split_index = tree.feature[0]
first_split_name = features[first_split_index]
first_split_value = tree.threshold[0]

print(f"First Split Feature: {first_split_name}")
print(f"Decision Boundary Value for First Split: {first_split_value}")


First Split Feature: concave_points1
Decision Boundary Value for First Split: 0.05127999931573868


In [194]:
# caculating gini
def gini(y):
    num = np.unique(y)
    gini = 1 - sum((np.sum(y == c) / len(y))**2 for c in num)
    return gini

# caculating entropy
def entropy(y):
    num = np.unique(y)
    entropy_value = -sum((np.sum(y == c) / len(y)) * np.log2(np.sum(y == c) / len(y)) for c in num)
    return entropy_value

# caculating misclass
def misclassific_error(y):
    y = y.to_numpy() 
    num = np.unique(y)
    classes = [np.sum(y == c) for c in num]
    return 1 - max(classes) / len(y)


In [196]:
split_column1 = clf.feature_importances_.argmax() 
split_value1 = clf.tree_.threshold[0]  

# caculate the aiming of the num of left and right
left_ind = X_train.iloc[:, split_column1] <= split_value1
right_ind = X_train.iloc[:, split_column1] > split_value1

left_y = y_train[left_ind]
right_y = y_train[right_ind]

In [198]:
# the whole num of gini entropy and mis_error
left_gini = gini(left_y)
right_gini = gini(right_y)
left_entropy = entropy(left_y)
right_entropy = entropy(right_y)
left_misclass = misclassific_error(left_y)
right_misclass = misclassific_error(right_y)

D:\software\conda\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [200]:
#the increasing of information
gini_acc = gini(y_train)
entropy_acc = entropy(y_train)
misclass_acc = misclassific_error(y_train)

weighted_left_gini = len(left_y) / len(y_train) * left_gini
weighted_right_gini = len(right_y) / len(y_train) * right_gini
weighted_left_entropy = len(left_y) / len(y_train) * left_entropy
weighted_right_entropy = len(right_y) / len(y_train) * right_entropy
weighted_left_misclass = len(left_y) / len(y_train) * left_misclass
weighted_right_misclass = len(right_y) / len(y_train) * right_misclass




D:\software\conda\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [202]:
weighted_gini=weighted_left_gini + weighted_right_gini
weighted_entropy=weighted_left_entropy + weighted_right_entropy
weighted_misclass=weighted_left_misclass + weighted_right_misclass

In [204]:
result_df = pd.DataFrame({
    'Metric': ['Gini', 'Entropy', 'Misclassification Error'],
    'Left Child': [float(left_gini), float(left_entropy), left_misclass],
    'Right Child': [float(right_gini), float(right_entropy), right_misclass],
    'Weighted Child': [float(weighted_gini), float(weighted_entropy), weighted_misclass],
    
    
})
pd.set_option('display.float_format', '{:.15f}'.format)

result_df

C:\Users\33615\AppData\Local\Temp\ipykernel_89428\719194449.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'Left Child': [float(left_gini), float(left_entropy), left_misclass],
C:\Users\33615\AppData\Local\Temp\ipykernel_89428\719194449.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'Right Child': [float(right_gini), float(right_entropy), right_misclass],
C:\Users\33615\AppData\Local\Temp\ipykernel_89428\719194449.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'Weighted Child': [float(weighted_gini), float(weighted_entropy), weighted_misclass],


,Metric,Left Child,Right Child,Weighted Child
0,Gini,0.107036869372768,0.204483945337298,0.144088175179062
1,Entropy,0.314355863592707,0.516599893360869,0.391253044141920
2,Misclassification Error,0.056737588652482,0.115606936416185,0.079120879120879


In [206]:
gini_gain = gini_acc - weighted_gini
entropy_gain = entropy_acc - weighted_entropy
misclass_gain = misclass_acc - weighted_misclass

In [208]:
result_df_gain = pd.DataFrame({
    'Metric': ['Gini Information Gain', 'Entropy Information Gain', 'Misclassification Error Information Gain'],
    'Gain': [float(gini_gain), float(entropy_gain), float(misclass_gain)] 
})

pd.set_option('display.float_format', '{:.25f}'.format)
result_df_gain


C:\Users\33615\AppData\Local\Temp\ipykernel_89428\14340117.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'Gain': [float(gini_gain), float(entropy_gain), float(misclass_gain)]  # 确保是 float 类型


,Metric,Gain
0,Gini Information Gain,0.3228506003311422123402963
1,Entropy Information Gain,0.5605096314929107759184035
2,Misclassification Error Information Gain,0.2923076923076923239186442
